# 수치데이터의 전처리

기계학습에 사용되는 많은 알고리즘들 예를 들어 [RBF kernel](https://en.wikipedia.org/wiki/Radial_basis_function_kernel) 기반의 [서포트벡터머신](https://en.wikipedia.org/wiki/Support_vector_machine)이나 [회귀분석](https://en.wikipedia.org/wiki/Linear_regression) 모델에서는 수치 데이터에 대해서 표준 정규 분포를 가정하고 있다. 즉, 데이터의 평균이 0 이고 분산이 1 인 정규분포를 따른다고 가정한다. 만일 주어진 데이터가 이러한 표준정규 분포가 아니라면 이를 표준정규 분포를 갖도록 변환해야 한다. 왜냐하면 데이터 분석에는 여러개의 변수 (특성값)이 사용될 수 있는데 이들의 절대 값이 임의의 분포를 갖는다면 절대치가 큰 변수에 따라 특성이 크게 영향을 받기 때문이다. 즉, 모든 변수의 변화 범위를 정규화 한 후에 데이터분석에 사용해야 한다. 

이 예에서는 주어진 수치 데이터를 일정한 비율로 조절하여 표준 정규 분포 등으로 재구성하는 방법을 소개하며 파이선이 제공하는 [sklearn](http://scikit-learn.org/)라이브러리를 사용하겠다. 이번 예에서는 태양의 흑점 데이터를 사용한다.

In [3]:
from data.load_sunspot_counts import load_sunspot_counts
data = load_sunspot_counts()
data = data[data['year'] >= 1850] 
# 값을 갖는 연도인 1850년 이후만 다룬다
counts = data['combined_count']
print(counts)

11688    253
11689    162
11690    217
11691     99
11692    108
11693    203
11694    142
11695     76
11696     95
11697    124
11698    162
11699    188
11700    146
11701    139
11702     65
11703    141
11704    124
11705    112
11706    101
11707     89
11708     78
11709     84
11710     87
11711    160
11712    270
11713    247
11714    222
11715    211
11716    199
11717    188
        ... 
72593     15
72594     36
72595     40
72596     41
72597     56
72598     59
72599     63
72600     69
72601     66
72602     64
72603     41
72604     41
72605     41
72606     36
72607     32
72608     25
72609     29
72610     30
72611     16
72612     28
72613     28
72614     15
72615     15
72616     17
72617     29
72618     25
72619     36
72620     23
72621     14
72622     13
Name: combined_count, dtype: int64


scikit-learn 라이브러리는 편리한 [scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html#sklearn.preprocessing.scale) 함수를 제공하는데 이는 각 데이터 세트의 크기를 원하는 방식으로 일정하게 스케일링해 준다.

In [4]:
from sklearn import preprocessing
counts_scaled = preprocessing.scale(counts)
print(counts_scaled)

[ 2.17027704  0.99315236  1.70460134 ..., -0.80487325 -0.92129217
 -0.93422761]


/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


이제 데이터의 평균이 0 이고 분산이 1 인지를 확인하겠다.

In [5]:
print(counts_scaled.mean(axis=0)) 
print(counts_scaled.std(axis=0))

8.58225081676e-17
1.0


평균이 8.58225081676e-17 인데 이는 0.0000000000000000858225081676 으로 거의 0이라고 하겠다. 분산은 1인 것을 확인하였다.
교차검증 데이터 처리 프로그램 예에서, 교차검증이 왜 유용한지를 설명하였다. 훈련용 데이터 뿐 아니라 테스트용 데이터도 반드시 스케일을 해두어야 한다. 경우에 따라서는 테스트 데이터를 미리 처리하는 것을 잊는 경우가 있는데 즉, 평균을 0 으로 바꾸고 분산을 1로 만드는 작업을 간혹 놓치는 경우가 있다. 또한 훈련용 데이터에 대하여 적용하는 것과 똑같은 방식으로 테스트용 데이터를 처리 해야 한다. 다행이 scikit-learn 라이브러리는 편리하게 이러한 기능을 [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) 으로 제공한다.

In [11]:
# 먼저 counts 두가지 세트로 나눈다
from sklearn.utils import shuffle
counts = shuffle(counts)
counts_train = counts[::2].values.reshape(-1, 1) # all even indices
counts_test = counts[1::2].values.reshape(-1, 1) # all odd indices 

# 훈련용 데이터에 대하여 스케일러를 설정한다
scaler = preprocessing.StandardScaler().fit(counts_train)

# 훈련용 데이터와 테스트 데이터를 모두 스케일링 한다
scaled_counts_train = scaler.transform(counts_train)
scaled_counts_test = scaler.transform(counts_test)

# 훈련용 데이터의 평균치와 분산이 각각 0 과 1 인지 확인한다
print(scaled_counts_train.mean(axis=0))
print(scaled_counts_train.std(axis=0))

# 테스트 데이터의 평균치와 분산이 각각 0 과 1 인지 확인한다
print(scaled_counts_test.mean(axis=0))
print(scaled_counts_test.std(axis=0))

[ -9.74815753e-17]
[ 1.]
[-0.00169314]
[ 0.99657483]


/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
